In [ ]:
import pandas as pd
import numpy as np
import os
import json

# Make sure the output directory exists
# This directory will store the generated HTML file
os.makedirs('../assets', exist_ok=True)

# Load data from CSV files
# Assuming these files are located in a 'data' directory one level up from the script
try:
    socioeconomic = pd.read_csv('../data/socioeconomic.csv')
    consumption = pd.read_csv('../data/Consumption.csv')
    regions = pd.read_csv('../data/Region.csv')
except FileNotFoundError as e:
    print(f"Error loading data file: {e}")
    print("Please ensure 'socioeconomic.csv', 'Consumption.csv', and 'Region.csv' are in the '../data/' directory.")
    # In a real script, you might handle this more gracefully, but for this example, we'll exit
    exit()

# Define translations for Danish to English socioeconomic statuses
# This is used for clearer labeling in the radar charts
socio_translations = {
    "Gennemsnitshusstand": "Average Household",
    "Selvstændig": "Self-employed",
    "Lønmodtager på højeste niveau": "High Income",
    "Lønmodtager på mellemniveau": "Medium Income",
    "Lønmodtager på grundniveau": "Basic Income",
    "Arbejdsløs": "Unemployed",
    "Uddannelsessøgende": "Student",
    "Pensionist, efterlønsmodtager": "Pensioner",
    "Ude af erhverv i øvrigt": "Not in Workforce"
}

# Clean and process socioeconomic data
# Renaming columns and adding a translated group column
socio_df = socioeconomic.copy()
socio_df.columns = [col.strip() for col in socio_df.columns] # Remove leading/trailing whitespace from column names
socio_df = socio_df.rename(columns={
    'Socioøkonomisk status': 'Group',
    '09.8 Pakkerejser': 'Packages',
    '11.1 Restaurationstjenester': 'Restaurants',
    '11.2 Overnatningsfaciliteter': 'Accommodation'
})

# Add translated names
socio_df['Group_EN'] = socio_df['Group'].map(socio_translations)

# Handle any remaining NaN values
socio_df = socio_df.dropna(subset=['Group_EN'])  # Remove rows where Group_EN is NaN

# Calculate total spending
socio_df['Total'] = socio_df['Packages'] + socio_df['Restaurants'] + socio_df['Accommodation']

# Process consumption/age data
age_df = consumption.copy()
# Find the exact column names containing 'Package Holidays - Fixed Prices', etc.
try:
    package_col_age = [col for col in age_df.columns if 'Package Holidays - Fixed Prices' in col][0]
    restaurant_col_age = [col for col in age_df.columns if 'Restaurant Services - Fixed Prices' in col][0]
    accommodation_col_age = [col for col in age_df.columns if 'Accommodation Services - Fixed Prices' in col][0]
except IndexError:
    print("Error: Could not find expected 'Fixed Prices' columns in Consumption.csv.")
    print("Please check column names like 'Package Holidays - Fixed Prices'.")
    exit()

age_df = age_df.rename(columns={
    package_col_age: 'Packages',
    restaurant_col_age: 'Restaurants',
    accommodation_col_age: 'Accommodation',
    'AGE': 'Age Group' # Assuming 'AGE' is the column for age groups
})
age_df['Total'] = age_df['Packages'] + age_df['Restaurants'] + age_df['Accommodation'] # Calculate total spending

# Process region data
region_df = regions.copy()
# Find the exact column names containing 'Package Holidays - Fixed Prices', etc.
try:
    package_col_region = [col for col in region_df.columns if 'Package Holidays - Fixed Prices' in col][0]
    restaurant_col_region = [col for col in region_df.columns if 'Restaurant Services - Fixed Prices' in col][0]
    accommodation_col_region = [col for col in region_df.columns if 'Accommodation Services - Fixed Prices' in col][0]
except IndexError:
    print("Error: Could not find expected 'Fixed Prices' columns in Region.csv.")
    print("Please check column names like 'Package Holidays - Fixed Prices'.")
    exit()

region_df = region_df.rename(columns={
    package_col_region: 'Packages',
    restaurant_col_region: 'Restaurants',
    accommodation_col_region: 'Accommodation',
    'REGION': 'Region' # Assuming 'REGION' is the column for regions
})
region_df['Total'] = region_df['Packages'] + region_df['Restaurants'] + region_df['Accommodation'] # Calculate total spending

# Define vibrant colors for each spending category
category_colors = {
    'Packages': '#9b59b6',      # Amethyst (Vibrant Purple)
    'Restaurants': '#2ecc71',   # Emerald (Vibrant Green)
    'Accommodation': '#f1c40f'  # Sunflower (Vibrant Yellow)
}

# Define ENGLISH labels for the legend
english_labels = {
    'Packages': 'Package Holidays',
    'Restaurants': 'Restaurants',
    'Accommodation': 'Accommodation'
}

# Find global maximum value across all datasets for consistent scale
global_max = max(
    socio_df[['Packages', 'Restaurants', 'Accommodation']].values.max(),
    age_df[['Packages', 'Restaurants', 'Accommodation']].values.max(),
    region_df[['Packages', 'Restaurants', 'Accommodation']].values.max()
)

# Function to prepare data for Chart.js format
def prepare_chart_data(df, group_col, display_col=None, normalize=True):
    if display_col is None:
        display_col = group_col
    
    # Get the list of groups
    groups = df[display_col].tolist()
    
    # Prepare datasets for each category
    datasets = []
    
    for category, color in category_colors.items():
        values = df[category].tolist()
        
        # Normalize values if required
        if normalize:
            values = [(val / global_max * 100) for val in values]
            
        # Create dataset for this category
        dataset = {
            'label': english_labels[category],
            'data': values,
            'backgroundColor': f"{color}80",  # Add transparency
            'borderColor': color,
            'borderWidth': 2,
            'pointBackgroundColor': color,
            'pointBorderColor': '#fff',
            'pointHoverBackgroundColor': '#fff',
            'pointHoverBorderColor': color
        }
        
        datasets.append(dataset)
    
    return {
        'labels': groups,
        'datasets': datasets
    }

# Prepare data for each chart
socioeconomic_data = prepare_chart_data(socio_df, 'Group', 'Group_EN')
age_data = prepare_chart_data(age_df, 'Age Group')
region_data = prepare_chart_data(region_df, 'Region')

# Create a dictionary with all chart data
chart_data = {
    'socioeconomic': socioeconomic_data,
    'age': age_data,
    'region': region_data
}

# Convert to JSON for use in the HTML
chart_data_json = json.dumps(chart_data)

# Create HTML template for the interactive radar chart
html_template = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Danish Travel Spending Patterns</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/3.9.1/chart.min.js"></script>
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f9f9f9;
        }}
        .chart-container {{
            position: relative;
            height: 70vh;
            width: 100%;
            margin: 20px auto;
            background-color: white;
            border-radius: 8px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            padding: 20px;
        }}
        .tab-container {{
            display: flex;
            justify-content: center;
            margin-bottom: 20px;
        }}
        .tab {{
            padding: 12px 24px;
            background-color: #e0e0e0;
            border: none;
            cursor: pointer;
            font-size: 16px;
            font-weight: 500;
            transition: all 0.3s ease;
            border-radius: 4px;
            margin: 0 8px;
        }}
        .tab.active {{
            background-color: #3498db;
            color: white;
            box-shadow: 0 2px 8px rgba(52, 152, 219, 0.4);
        }}
        .tab:hover:not(.active) {{
            background-color: #d0d0d0;
        }}
        h1, h2 {{
            text-align: center;
            color: #333;
        }}
        .subtitle {{
            text-align: center;
            color: #666;
            margin-bottom: 30px;
            font-style: italic;
        }}
        .insight-highlight {{
            background: linear-gradient(90deg, #3498db22 0%, #3498db11 100%);
            border-radius: 8px;
            padding: 20px;
            margin: 30px 0;
        }}
        @media (max-width: 768px) {{
            .tab {{
                padding: 8px 16px;
                font-size: 14px;
            }}
        }}
    </style>
</head>
<body>
    <h1>Danish Travel Spending Patterns</h1>
    <p class="subtitle">Explore how different Danish demographics allocate their travel budgets</p>
    
    <div class="tab-container">
        <button class="tab active" onclick="showChart('socioeconomic')">Socioeconomic Groups</button>
        <button class="tab" onclick="showChart('age')">Age Groups</button>
        <button class="tab" onclick="showChart('region')">Regions</button>
    </div>
    
    <div class="chart-container">
        <canvas id="radarChart"></canvas>
    </div>

    <div class="insight-highlight">
        <p>Restaurant spending leads across all demographics, ages, and regions. Students allocate more to accommodation while older travelers favor package deals, but these are minor variations within the broader restaurant-centric framework. The consistency suggests this preference is both generational and cultural, deeply embedded in Danish society regardless of life stage.</p>
    </div>

    <script>
        // Store chart instance for updating
        let radarChart;
        
        // Chart data from Python
        const chartData = {chart_data_json};
        
        // Chart configuration
        const config = {{
            type: 'radar',
            data: chartData.socioeconomic, // Initial dataset
            options: {{
                responsive: true,
                maintainAspectRatio: false,
                plugins: {{
                    legend: {{
                        position: 'top',
                        labels: {{
                            font: {{
                                size: 14
                            }},
                            padding: 20
                        }}
                    }},
                    title: {{
                        display: true,
                        text: 'Socioeconomic Groups: Travel Spending Distribution',
                        font: {{
                            size: 18,
                            weight: 'bold'
                        }},
                        padding: {{
                            top: 10,
                            bottom: 30
                        }}
                    }},
                    tooltip: {{
                        callbacks: {{
                            label: function(context) {{
                                return ` ${{context.dataset.label}}: ${{context.raw.toFixed(1)}}%`;
                            }}
                        }}
                    }}
                }},
                scales: {{
                    r: {{
                        angleLines: {{
                            color: '#cccccc'
                        }},
                        grid: {{
                            color: '#cccccc'
                        }},
                        pointLabels: {{
                            font: {{
                                size: 14
                            }},
                            color: '#555555'
                        }},
                        ticks: {{
                            backdropColor: 'transparent',
                            color: '#777777',
                            showLabelBackdrop: false,
                            font: {{
                                size: 10
                            }}
                        }},
                        suggestedMin: 0,
                        suggestedMax: 100
                    }}
                }},
                elements: {{
                    line: {{
                        tension: 0.1 // Smoother lines
                    }}
                }}
            }}
        }};

        // Initialize the chart
        window.onload = function() {{
            const ctx = document.getElementById('radarChart').getContext('2d');
            radarChart = new Chart(ctx, config);
        }};

        // Function to switch between datasets
        function showChart(category) {{
            const titles = {{
                socioeconomic: 'Socioeconomic Groups: Travel Spending Distribution',
                age: 'Age Groups: Travel Spending Distribution',
                region: 'Regions: Travel Spending Distribution'
            }};
            
            // Update chart data
            radarChart.data = chartData[category];
            
            // Update chart title
            radarChart.options.plugins.title.text = titles[category];
            
            // Update chart
            radarChart.update();
            
            // Update active tab
            document.querySelectorAll('.tab').forEach(tab => {{
                tab.classList.remove('active');
            }});
            event.currentTarget.classList.add('active');
        }}
    </script>
</body>
</html>
'''.replace('{JSON_DATA}', chart_data_json)

# Write the HTML to a file
output_file = '../assets/danish_travel_radar_interactive.html'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(html_template)

print(f"Interactive radar visualization created successfully as '{output_file}'!")

NameError: name 'JSON_DATA' is not defined